In [6]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [115]:
class LogisticRegression:
    
    #intialize model parametersc
    def __init__ (self,learning_rate, batch_size, n_epochs):
        
        self.learning_rate = learning_rate
        self.batch_size =batch_size
        self.n_epochs = n_epochs
    
    def ReadData(self,PATH):
        
        self.MNIST = input_data.read_data_sets(PATH, one_hot=True)
        
    def DecInput(self):
        
        self.X =  tf.placeholder(tf.float32, shape =[self.batch_size,784])
        self.Y =  tf.placeholder(tf.float32, shape =[self.batch_size,10])

    def DecVariables(self):
        
        self.W = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01), name="weights")
        self.b = tf.Variable(tf.zeros([1, 10]), name="bias")
        self.logits = tf.matmul(self.X, self.W) + self.b
    
    def _Loss(self):
        
        self.entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = self.Y,logits = self.logits)
        self.loss = tf.reduce_mean(self.entropy)
        
    def _create_optimizer(self):
        
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate).minimize(self.loss)
    
    def _Build_Graph(self):
        
        self.DecInput()
        self.DecVariables()
        self._Loss()
        self._create_optimizer()
    
    def buildSum(self,sess):
        
        n_batches = int(self.MNIST.test.num_examples/self.batch_size)
        self.total_correct_preds = 0
        for i in range(n_batches):
            X_batch, Y_batch = self.MNIST.test.next_batch(batch_size)
            _, logits_batch = sess.run([self.optimizer, self.logits],
            feed_dict={self.X: X_batch, self.Y:Y_batch})
            preds = tf.nn.softmax(logits_batch)
            correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
            accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) 
            self.total_correct_preds += sess.run(accuracy)
        
        return self.total_correct_preds

def _train(model):

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    n_batches = int(model.MNIST.train.num_examples/model.batch_size)
    for i in range(model.n_epochs): # train the model n_epochs times
        for _ in range(n_batches):
            X_batch, Y_batch = model.MNIST.train.next_batch(model.batch_size)
            sess.run(model.optimizer, feed_dict={model.X: X_batch, model.Y:Y_batch})
    
    acc = model.buildSum(sess)
    return acc

In [111]:
learning_rate = 0.01
batch_size = 128
n_epochs = 25
model = LogisticRegression(learning_rate, batch_size, n_epochs)

In [112]:
model.ReadData("/data/mnist")

Extracting /data/mnist\train-images-idx3-ubyte.gz
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


In [113]:
model._Build_Graph()

In [116]:
acc = _train(model)